In [7]:
!pip install ftfy
!pip install nltk
import warnings
warnings.filterwarnings("ignore")
import ftfy
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re

from math import exp
from numpy import sign

from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk import PorterStemmer

from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [13]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-03-21 06:08:03--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.227.155
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.227.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  75.9MB/s    in 25s     

2021-03-21 06:08:28 (64.1 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [14]:
!pip install gensim
from gensim.models import KeyedVectors

In [22]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
depressive_tweets = 'depressive_tweets_processed.csv'
random_tweets = 'Sentiment Analysis Dataset 2.csv'
embedding_file = '/root/input/GoogleNews-vectors-negative300.bin.gz'

In [17]:
depressive_tweets_df = pd.read_csv(depressive_tweets, sep = '|', header = None, usecols = range(0,9), nrows = 3200)
random_tweets_df = pd.read_csv(random_tweets, encoding = "ISO-8859-1", usecols = range(0,4), nrows = 12000)

***Preprocessing the Data***

In [18]:
word2vec = KeyedVectors.load_word2vec_format(embedding_file, binary=True)

In [19]:
 # Expand Contraction
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [20]:
def clean_tweets(tweets):
    cleaned_tweets = []
    for tweet in tweets:
        tweet = str(tweet)
        # if url links then dont append to avoid news articles
        # also check tweet length, save those > 10 (length of word "depression")
        if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
            #remove hashtag, @mention, emoji and image URLs
            tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", tweet).split())
            
            #fix weirdly encoded texts
            tweet = ftfy.fix_text(tweet)
            
            #expand contraction
            tweet = expandContractions(tweet)

            #remove punctuation
            tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())

            #stop words
            stop_words = set(stopwords.words('english'))
            word_tokens = nltk.word_tokenize(tweet) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            tweet = ' '.join(filtered_sentence)

            #stemming words
            tweet = PorterStemmer().stem(tweet)
            
            cleaned_tweets.append(tweet)

    return cleaned_tweets

In [23]:
depressive_tweets_arr = [x for x in depressive_tweets_df[5]]
random_tweets_arr = [x for x in random_tweets_df['SentimentText']]
X_d = clean_tweets(depressive_tweets_arr)
X_r = clean_tweets(random_tweets_arr)

***Tokenizing and Padding the Data***

In [24]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_d + X_r)

In [25]:
sequences_d = tokenizer.texts_to_sequences(X_d)
sequences_r = tokenizer.texts_to_sequences(X_r)

In [26]:
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 21548 unique tokens


In [27]:
data_d = pad_sequences(sequences_d, maxlen=140)
data_r = pad_sequences(sequences_r, maxlen=140)
print('Shape of data_d tensor:', data_d.shape)
print('Shape of data_r tensor:', data_r.shape)

Shape of data_d tensor: (2308, 140)
Shape of data_r tensor: (11911, 140)


***Embedding Matrix***

In [30]:
nb_words = min(20000, len(word_index))

embedding_matrix = np.zeros((nb_words, 300))

for (word, idx) in word_index.items():
    if word in word2vec.vocab and idx < 20000:
        embedding_matrix[idx] = word2vec.word_vec(word)

***Splitting the data into traning and testing set***

In [33]:
# Assigning labels to the depressive tweets and random tweets data
labels_d = np.array([1] * 3200)
labels_r = np.array([0] * 12000)

# Splitting the arrays into test (60%), validation (20%), and train data (20%)
perm_d = np.random.permutation(len(data_d))
idx_train_d = perm_d[:int(len(data_d)*(0.6))]
idx_test_d = perm_d[int(len(data_d)*(0.6)):int(len(data_d)*(0.6+0.2))]
idx_val_d = perm_d[int(len(data_d)*(0.6+0.2)):]

perm_r = np.random.permutation(len(data_r))
idx_train_r = perm_r[:int(len(data_r)*(0.6))]
idx_test_r = perm_r[int(len(data_r)*(0.6)):int(len(data_r)*(0.6+0.2))]
idx_val_r = perm_r[int(len(data_r)*(0.6+0.2)):]

# Combine depressive tweets and random tweets arrays
data_train = np.concatenate((data_d[idx_train_d], data_r[idx_train_r]))
labels_train = np.concatenate((labels_d[idx_train_d], labels_r[idx_train_r]))
data_test = np.concatenate((data_d[idx_test_d], data_r[idx_test_r]))
labels_test = np.concatenate((labels_d[idx_test_d], labels_r[idx_test_r]))
data_val = np.concatenate((data_d[idx_val_d], data_r[idx_val_r]))
labels_val = np.concatenate((labels_d[idx_val_d], labels_r[idx_val_r]))

# Shuffling
perm_train = np.random.permutation(len(data_train))
data_train = data_train[perm_train]
labels_train = labels_train[perm_train]
perm_test = np.random.permutation(len(data_test))
data_test = data_test[perm_test]
labels_test = labels_test[perm_test]
perm_val = np.random.permutation(len(data_val))
data_val = data_val[perm_val]
labels_val = labels_val[perm_val]

***Building the model***

In [53]:
model = Sequential()
# Embedded layer
model.add(Embedding(len(embedding_matrix), 300, weights=[embedding_matrix], 
                            input_length=140, trainable=False))
# Convolutional Layer
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
# LSTM Layer
model.add(LSTM(300))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [54]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 140, 300)          6000000   
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 140, 32)           28832     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 70, 32)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 70, 32)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               399600    
_________________________________________________________________
dropout_5 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                

***Traning the Model***

In [55]:
early_stop = EarlyStopping(monitor='acc', patience=3)

hist = model.fit(data_train, labels_train, validation_data=(data_val, labels_val), epochs=20, batch_size=40, shuffle=True, callbacks=[early_stop])

Epoch 1/20
214/214 [==============================] - 5s 13ms/step - loss: 0.2819 - acc: 0.9067 - val_loss: 0.0600 - val_acc: 0.9863
Epoch 2/20
214/214 [==============================] - 2s 11ms/step - loss: 0.0391 - acc: 0.9921 - val_loss: 0.0591 - val_acc: 0.9866
Epoch 3/20
214/214 [==============================] - 2s 11ms/step - loss: 0.0324 - acc: 0.9923 - val_loss: 0.0644 - val_acc: 0.9873
Epoch 4/20
214/214 [==============================] - 2s 11ms/step - loss: 0.0256 - acc: 0.9938 - val_loss: 0.0575 - val_acc: 0.9873
Epoch 5/20
214/214 [==============================] - 2s 11ms/step - loss: 0.0532 - acc: 0.9848 - val_loss: 0.0612 - val_acc: 0.9863
Epoch 6/20
214/214 [==============================] - 2s 11ms/step - loss: 0.0333 - acc: 0.9913 - val_loss: 0.0553 - val_acc: 0.9870
Epoch 7/20
214/214 [==============================] - 2s 11ms/step - loss: 0.0246 - acc: 0.9941 - val_loss: 0.0708 - val_acc: 0.9880
Epoch 8/20
214/214 [==============================] - 2s 11ms/step - 

***Tessting the Model***

In [57]:
pred = model.predict(data_test)
pred = np.round(pred.flatten())
accuracy = accuracy_score(labels_test, pred)
print("Accuracy: %.2f%%" % (accuracy*100))

Accuracy: 99.09%


In [59]:
print(classification_report(labels_test, pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2382
           1       0.98      0.96      0.97       462

    accuracy                           0.99      2844
   macro avg       0.99      0.98      0.98      2844
weighted avg       0.99      0.99      0.99      2844

